In [1]:
import pandas as pd
from io import StringIO
import requests

def getCurie_Disease(name):
    itemRequest = 'https://name-resolution-sri.renci.org/lookup?string=' + name + '&autocomplete=false&offset=0&limit=10&biolink_type=DiseaseOrPhenotypicFeature'
    returned = (pd.read_json(StringIO(requests.get(itemRequest).text)))
    resolvedName = returned.curie
    resolvedLabel = returned.label
    return resolvedName, resolvedLabel

def getCurie_Drug(name):
    itemRequest = 'https://name-resolution-sri.renci.org/lookup?string=' + name + '&autocomplete=false&offset=0&limit=10&biolink_type=ChemicalOrDrugOrTreatment'
    returned = (pd.read_json(StringIO(requests.get(itemRequest).text)))
    resolvedName = returned.curie
    resolvedLabel = returned.label
    return resolvedName, resolvedLabel

def build_string_from_list(list):
    outString = "["
    for item in list:
        outString += item + ", "
    outString = outString[:-2] + "]"
    return outString

diseaseData = pd.read_excel('../drugs_to_diseases_test_2.xlsx')
diseaseLabelList = []
diseaseIDList = []
diseaseList = []
drugList = []
drugIDList = []
drugLabelList = []

nRows = len(list(diseaseData['diseases treated']))

labelDict = {}
idDict = {}

for index, row in diseaseData.iterrows():
    if index % 100 == 0:
        print("Progress: ", index / nRows * 100, "%")
    curr_row_diseasesTreated = row['diseases treated']
    if type(curr_row_diseasesTreated)!=float:
        
        curr_row_drugsInTherapy = row['active ingredient(s)']
        curr_row_disease_ids = []
        curr_row_disease_id_labels = []
        curr_row_diseaseList = curr_row_diseasesTreated.replace("[","").replace("]","").replace('\'','').split(',')
        #print("disease list: ", curr_row_diseaseList)
        
        try:
            drugCurie,drugLabel = getCurie_Drug(curr_row_drugsInTherapy)
            drugID = drugCurie[0]
            drugIDLabel = drugLabel[0]
                
        except:
            print("could not identify drug: ", curr_row_drugsInTherapy)
            drugID = "NameRes Failed"
            drugIDLabel = "NameRes Failed"

        for idx2,item in enumerate(curr_row_diseaseList):
            item = item.strip().upper().replace(" \n","").replace(" (PREVENTATIVE)","")
            curr_row_diseaseList[idx2] = item
            #print(item)
            try:
                #print(item)
                diseaseCurie,diseaseLabel = getCurie_Disease(item)
                diseaseIDList.append(diseaseCurie[0])
                diseaseLabelList.append(diseaseLabel[0])
                diseaseList.append(item)
                drugList.append(curr_row_drugsInTherapy)
                drugIDList.append(drugID)
                drugLabelList.append(drugIDLabel)
                
            except:
                print("error during name resolving")



sheetData = pd.DataFrame(data=[diseaseIDList, diseaseLabelList, diseaseList, drugList, drugIDList, drugLabelList]).transpose()
sheetData.columns = ['disease IDs', 'disease ID labels', 'list of diseases', 'active ingredients in therapy', 'drug ID', 'drug ID Label']
print(sheetData)
sheetData.to_excel("indication-list-fda-v1.xlsx")


Progress:  0.0 %
error during name resolving
Progress:  0.20251934059702703 %
Progress:  0.40503868119405406 %
Progress:  0.6075580217910811 %
Progress:  0.8100773623881081 %
error during name resolving
Progress:  1.012596702985135 %
Progress:  1.2151160435821622 %
error during name resolving
Progress:  1.417635384179189 %
error during name resolving
Progress:  1.6201547247762162 %
Progress:  1.8226740653732432 %
Progress:  2.02519340597027 %
Progress:  2.2277127465672972 %
error during name resolving
could not identify drug:  {'QUERCUS MACROCARPA POLLEN', 'EQUUS CABALLUS HAIR', 'RUMEX ACETOSELLA POLLEN', 'MUS MUSCULUS SKIN', 'AMARANTHUS SPINOSUS POLLEN', 'KOELERIA MACRANTHA POLLEN', 'ASPERGILLUS NIGER VAR. NIGER', 'ANAS PLATYRHYNCHOS FEATHER', 'BLATELLA GERMANICA', 'BACCHARIS HALIMIFOLIA POLLEN', 'PAECILOMYCES VARIOTII', 'CONCORD GRAPE', 'COCHLIOBOLUS SPICIFER', 'FUSARIUM OXYSPORUM VASINFECTUM', 'QUERCUS ALBA POLLEN', 'TRITICUM AESTIVUM POLLEN', 'ENGLISH WALNUT', 'CEIBA PENTANDRA FIBE

In [ ]:
sheetData = pd.DataFrame(data=[diseaseIDList, diseaseLabelList, diseaseList, drugList, drugIDList, drugLabelList]).transpose()
sheetData.columns = ['disease IDs', 'disease ID labels', 'list of diseases', 'active ingredients in therapy', 'drug ID', 'drug ID Label']
print(sheetData)
sheetData.to_excel("indication-list-fda-v1.xlsx")